# Storing skymaps

## Read raw skymap files

In [65]:
import pickle
from pathlib import Path
import math
import numpy as np
from lsst.sphgeom import Box, ConvexPolygon, LonLat, UnitVector3d
import yaml
from skymap_to_poly_coords import load_pickle_skymap


In [66]:
package_root = Path.home() / "skymap-to-poly-coords"

raw_skymaps_dir = package_root / "tests" / "data" / "raw_skymaps"
skymap_path = raw_skymaps_dir / "skyMap_lsst_cells_v1_skymaps.pickle"

skymap_out_dir = package_root / "skymaps_out"
inner_poly_path = skymap_out_dir / "inner_polygons.yaml"
outer_poly_path = skymap_out_dir / "outer_polygons.yaml"

inner_poly_ring_optimized_path = skymap_out_dir / "inner_polygons_ring_optimized.yaml"
outer_poly_ring_optimized_path = skymap_out_dir / "outer_polygons_ring_optimized.yaml"

In [67]:
lsst_skymap = load_pickle_skymap(skymap_path)
lsst_skymap

## A note on rings sky map pixelization 
*From the RingsSkyMap docstring in lsst.skymap:*

We divide the sphere into N rings of Declination, plus the two polar
caps, which sets the size of the individual tracts.  The rings are
divided in RA into an integral number of tracts of this size; this
division is made at the Declination closest to zero so as to ensure
full overlap.

Rings are numbered in the rings from south to north. The south pole cap is
``tract=0``, then the tract at ``raStart`` in the southernmost ring is
``tract=1``. Numbering continues (in the positive RA direction) around that
ring and then continues in the same fashion with the next ring north, and
so on until all reaching the north pole cap, which is
``tract=len(skymap) - 1``.

However, ``version=0`` had a bug in the numbering of the tracts: the first
and last tracts in the first (southernmost) ring were identical, and the
first tract in the last (northernmost) ring was missing. When using
``version=0``, these tracts remain missing in order to preserve the
numbering scheme.

## Storage options


We would like to support both inner and outer polygons for the tracts.  

Tracts are arranged in "rings", which span horizontal regions of the sky.

→ Inner polys:
- represent the exact boundaries of a given tract
- do not overlap

→ Outer polys:
- may (will?) overlap with adjacent tracts' outer polys
- are akin to pixels + margins in HATS catalogs

## Option 1: Two YAML files (inner polys and outer polys) that store each tracts' corners

Basically, a brute force approach where we calculate each tracts' vertices in radec and dump them into a yaml.

#### Inner polys/outer polys:
- Store per-tract: `ra_min`, `ra_max`, `dec_min`, `dec_max`
- Requires 4 floats per tract, so for ~19k tracts:
  4 × 8 B × 19,000 ≈ ~600 KB (before YAML overhead; ~1–1.5 MB total)
- However, could compress to a `.npz` of around ~0.5–0.8 MB
- **Just kidding. Estimates were a bit off; a single file is about 4 MiB**

| Pros     | Cons |
| -------- | ------- |
| Easy to understand and verify  | Disk usage   |
| Doesn’t require any LSST WCS machinery to read the yaml file | Less precise because of floating points? Is this a problem?   |
|  | Storing patches is gonna go crazy on this. Better to iterate to a more performant version for that.  |


<!-- #### Pros
- Easy to understand and verify
- Doesn’t require any LSST WCS machinery to read the yaml file

#### Cons
- Disk usage
- Less precise because of floating points? Is this a problem?
- Storing patches is gonna go crazy on this. Better to iterate to a more performant version for that.
  - Otherwise, we'll have to break these files up into a series of yamls to accomodate patches. -->

### Write inner_poly and outer_poly files

In [68]:
from skymap_to_poly_coords import write_polygons_ra_dec


write_polygons_ra_dec(lsst_skymap, inner_poly_path, inner=True)
#write_polygons_ra_dec(lsst_skymap, outer_poly_path, inner=False)

## Option 2: Reconstruct everything using projection + geometry

Can we do this without using LSST geom and mucking around with WCS packages and some really heavy projection/reprojection code?

I don't think re-implementing LSST's skymap package is in scope for this task, but open to suggestions if someone has an idea.

## Option 3: Hybrid; store per-ring Dec data, and per-tract RA data
### Similar principle with patches--each row of patches will act as its own "patch ring" with shared Dec

Can we make this assumption, or does it break down with projection distortions near the poles, rounding errors, or other confounding factors?

### Let's confirm two things:
1. Are the dec_min and dec_max values exactly the same for all tracts in a given ring?
2. Can each tract polygon be reconstructed from just the ring’s declination bounds and the tract’s RA vertex list?

### Assumption 1: dec_min and dec_max are exactly the same for all tracts in a ring (✓)
(within floating point imprecision)

In [69]:
import yaml
from pathlib import Path

yaml_path = inner_poly_path  # todo check outers also

# Load YAML
with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

# Parse and sort tracts
tract_items = sorted(data["tracts"].items(), key=lambda x: int(x[0]))

rings = []
current_ring = []
last_dec_bounds = None

for tract_id_str, entry in tract_items:
    tract_id = int(tract_id_str)
    polygon = entry.get("polygon")

    if polygon is None:
        print(f"Skipping degenerate tract {tract_id} with no polygon data.")
        continue

    decs = [point[1] for point in polygon]

    # Collect rounded decs to eliminate fp imprecision
    unique_decs = {round(d, 12) for d in decs}
    if len(unique_decs) > 2:
        raise ValueError(f"⚠️ Tract {tract_id} has {len(unique_decs)} unique Decs: {sorted(unique_decs)}")

    # Generate bounds for declination
    dec_min = round(min(decs), 12)
    dec_max = round(max(decs), 12)
    dec_bounds = (dec_min, dec_max)

    if last_dec_bounds is None:
        # First ring
        last_dec_bounds = dec_bounds

    if dec_bounds != last_dec_bounds:
        # New ring starts
        rings.append((last_dec_bounds, current_ring))
        current_ring = [tract_id]
        last_dec_bounds = dec_bounds
    else:
        current_ring.append(tract_id)

# Append the last ring
if current_ring:
    rings.append((last_dec_bounds, current_ring))

# Report
verbose = False
if verbose:
    for i, (bounds, tracts) in enumerate(rings):
        print(f"→ Ring {i}: dec_min/max = {bounds}, tracts = {tracts[0]} to {tracts[-1]}, total = {len(tracts)}")
else:
    print(f"→ Found {len(rings)} rings with {sum(len(r[1]) for r in rings)} tracts.")

→ Found 122 rings with 18938 tracts.


### Assumption 2: Each tract polygon can be reconstructed from just the ring’s declination bounds and the tract’s RA vertex list

This assumption relies on a couple other questions:
1. How many unique RA values are there per-tract? Always just 2, or sometimes 3 or 4?
2. Is there a patern to what order the polygon vertices are listed? Is it, say, always the 
  upper-left and clockwise around?

#### 2.1 How many unique RA values are in a tract?

In [70]:
from collections import Counter

ra_counts = Counter()

for tract_id_str, entry in data["tracts"].items():
    polygon = entry.get("polygon")
    if polygon is None:
        continue

    ras = [round(point[0], 10) for point in polygon]
    unique_ras = set(ras)
    ra_counts[len(unique_ras)] += 1

    if len(unique_ras) == 3:
        print(f"⚠️ Tract {tract_id_str} has 3 unique RA values: {unique_ras}")

print("RA value counts per tract:")
for count, num_tracts in sorted(ra_counts.items()):
    print(f"  {count} unique RA values: {num_tracts} tracts")


RA value counts per tract:
  1 unique RA values: 2 tracts
  2 unique RA values: 18936 tracts


#### 2.2 Do most tracts follow the same curve orientation? If so, is it clockwise or counter-clockwise?

In [71]:
import math

def unwrap_ra_sequence(ras):
    """Unwrap RA values to make them monotonic, accounting for 0/360 crossover."""
    unwrapped = [ras[0]]
    for ra in ras[1:]:
        prev = unwrapped[-1]
        if ra < prev - 180:
            ra += 360
        elif ra > prev + 180:
            ra -= 360
        unwrapped.append(ra)
    return unwrapped

def polygon_orientation_ra_dec(points):
    """Return 'CW' or 'CCW' for a polygon defined by RA/Dec points.

    Parameters
    ----------
    points : list of (float, float)
        List of (RA, Dec) pairs in degrees. Polygon is assumed to be closed
        (first and last point the same) or will be treated as such.

    Returns
    -------
    str
        "CW" for clockwise, "CCW" for counter-clockwise, "Err" if not enough points.
    """
    if len(points) < 3:
        return "ERR"

    ras, decs = zip(*points)
    ras_unwrapped = unwrap_ra_sequence(ras)
    unwrapped_points = list(zip(ras_unwrapped, decs))

    # Shoelace formula: sum over (x_i * y_{i+1} - x_{i+1} * y_i)
    area = 0.0
    n = len(unwrapped_points)
    for i in range(n):
        x0, y0 = unwrapped_points[i]
        x1, y1 = unwrapped_points[(i + 1) % n]
        area += (x0 * y1 - x1 * y0)

    orientation = "CCW" if area > 0 else "CW"
    return orientation


In [72]:
from collections import Counter

ordering_counter = Counter()
clockwise_tracts = []

for tract_id_str, entry in data["tracts"].items():
    polygon = entry.get("polygon")
    if polygon is None or len(polygon) != 4:
        continue

    order = polygon_orientation_ra_dec(polygon)
    ordering_counter[order] += 1

    if order == "CW":
        clockwise_tracts.append(int(tract_id_str))

print("Polygon vertex ordering:")
for k, v in ordering_counter.items():
    print(f"  {k}: {v} tracts")

print()
print(f"{len(clockwise_tracts)} tracts are clockwise: {', '.join(map(str, clockwise_tracts))}")


Polygon vertex ordering:
  CW: 2 tracts
  CCW: 18936 tracts

2 tracts are clockwise: 0, 18937


#### 2.3 Do most tracts list their vertices starting from a certain corner? Which corner is that?

In [73]:
def classify_starting_corner(polygon):
    """Classify the starting corner of a polygon based on unwrapped RA/Dec.

    Parameters
    ----------
    polygon : list of (float, float)
        List of (RA, Dec) pairs in degrees. Assumed to be in polygon vertex order.

    Returns
    -------
    str
        One of: "top-left", "top-right", "bottom-left", "bottom-right"
    """
    if len(polygon) < 3:
        raise ValueError("Need at least 3 vertices to classify corners")

    # Unwrap RAs to make comparisons safe
    ras, decs = zip(*polygon)
    unwrapped_ras = unwrap_ra_sequence(ras)
    start_ra, start_dec = unwrapped_ras[0], decs[0]

    min_ra, max_ra = min(unwrapped_ras), max(unwrapped_ras)
    min_dec, max_dec = min(decs), max(decs)

    horiz = "left" if abs(start_ra - min_ra) < 1e-6 else "right"
    vert = "top" if abs(start_dec - max_dec) < 1e-6 else "bottom"

    return f"{vert}-{horiz}"


In [74]:
starting_corner_counter = Counter()

for tract_id_str, entry in data["tracts"].items():
    polygon = entry.get("polygon")
    if polygon is None or len(polygon) != 4:
        continue

    corner = classify_starting_corner(polygon)
    starting_corner_counter[corner] += 1

print("Starting corner classification:")
for corner, count in starting_corner_counter.items():
    print(f"  {corner}: {count} tracts")

Starting corner classification:
  bottom-left: 18938 tracts


#### Things we now know about our input skymap:
- All tracts start listing their vertices at the **bottom left** corner
- Almost all tracts list their vertices in **counter-clockwise** order--except the poles, who nominally use clockwise order, but...
- Almost all tracts have only 2 unique RA values; however:
  - 32 tracts have 3 unique RA values
  - 2 tracts have only 1 unique RA value (these are likely the poles)

So, we will follow the same convention for how we list our RA values in our ring-optimized poly coord files:
- Always start with the RA of the **bottom left** corner
- In most cases: list the 2 unique RA values (following the bottom-left rule, this will be the left-most RA, then the right-most)
- When we have 3 unique RA values, simply list all 4 RA values that are present, in **counter-clockwise** order
- *The poles will be weird. Still not sure the best way to encode them.*

### Check assumption: are all RAs at equal intervals within a ring?

In [75]:
import math
from collections import defaultdict

lsst_ring_nums = lsst_skymap._ringNums

def get_ring_num(tract_id, ring_nums):
    # Given an int tract_id and 1D array of ring numbers, such that ring_nums[3] = 4 means there 
    # are 4 tracts in ring 3, return the ring number for the tract.
    # Note that tract_id 0 is the pole, and has ring -1. Ring 0 starts at tract_id 1.

    # Invalid tract_id check.
    if tract_id < 0 or tract_id > sum(ring_nums) + 1:
        raise ValueError(f"Invalid tract_id {tract_id} for ring_nums with length {len(ring_nums)} "
                         f"and sum {sum(ring_nums)}")

    # Add the poles (but note, we'll have to subtract 1 later to get the correct ring number).
    ring_nums = [1] + ring_nums  # Add the south pole as ring -1 with 1 tract.
    ring_nums.append(1)  # Add the north pole as last ring.

    # Find the ring number by subtracting the number of tracts in each ring.
    ring_num = 0
    for num in ring_nums:
        if tract_id < num:
            return ring_num - 1  # Subtract 1 to account for the added south pole.
        tract_id -= num
        ring_num += 1

    return -1  # Should not happen if tract_id is valid


def get_ra_interval(tract_id, ring_num, entry):
    """Get the RA interval of a polygon, handling unwrapped RAs."""

    # Get the polygon for the tract. - does this handle poles? i think so.
    polygon = entry.get("polygon")
    if polygon is None:
        return -1.0

    # Figure out the ring number for this tract.
    ring_num = get_ring_num(tract_id, lsst_ring_nums)

    # Get the RAs.
    ras = [round(point[0], 10) for point in polygon]
    unwrapped_ras = unwrap_ra_sequence(ras)
    unique_ras = set(unwrapped_ras)

    # Edge cases and errors.
    if len(unique_ras) < 2:
        return -1.0
    elif len(unique_ras) > 2:
        raise ValueError(f"Polygon has {len(unique_ras)} unique RA values: {unique_ras}")
    
    # Return the interval between the two unique RAs.
    sorted_ras = sorted(unique_ras)
    return (sorted_ras[1] - sorted_ras[0])


In [76]:
intervals = {}
for tract_id_str, entry in data["tracts"].items():
    # Calculate the ring number and RA interval for each tract.
    ring_num = get_ring_num(int(tract_id_str), lsst_ring_nums)
    new_interval = get_ra_interval(int(tract_id_str), ring_num, entry)

    # Record the interval for this ring number.
    if intervals.get(ring_num) is None:
        intervals[ring_num] = new_interval
    else:
        # Check if the new interval matches the existing one for this ring.
        if not math.isclose(new_interval, intervals[ring_num], abs_tol=1e-8):
            raise ValueError(f"Interval for tract {tract_id_str} in ring {ring_num} "
                             f"differs from previous: {new_interval} vs {intervals[ring_num]}")

# Print the intervals for each ring
for ring_num, interval in sorted(intervals.items()):
    if ring_num % 10 == 0:
        print(f"Ring {ring_num}: RA interval = {interval:.6f} degrees")
print(f"Total rings: {len(intervals)}; unique intervals: {len(set(intervals.values()))}")

Ring 0: RA interval = 36.000000 degrees
Ring 10: RA interval = 5.000000 degrees
Ring 20: RA interval = 2.790698 degrees
Ring 30: RA interval = 2.033898 degrees
Ring 40: RA interval = 1.682243 degrees
Ring 50: RA interval = 1.525424 degrees
Ring 60: RA interval = 1.481481 degrees
Ring 70: RA interval = 1.538462 degrees
Ring 80: RA interval = 1.706161 degrees
Ring 90: RA interval = 2.080925 degrees
Ring 100: RA interval = 2.926829 degrees
Ring 110: RA interval = 5.454545 degrees
Ring 120: RA interval = -1.000000 degrees
Total rings: 122; unique intervals: 58


### Check: Is RA start the same for all rings?
If these two things are true, we can optimize the ra storage :)

In [77]:
lsst_skymap.config.raStart

0.0

Seems to imply this is skymap-wide. Nice! We can procede with RA-optimization, too.

## Rewrite code from Option 1 to be ring-aware

In [78]:
# def write_ring_optimized_polygons_ra_dec(skymap, output_path, inner=True):
#     """Write tract polygons in RA/Dec format to a YAML file, grouped by ring and optimized
#     by storing declination bounds per ring and RA values per tract. Polar caps are stored separately.

#     Parameters
#     ----------
#     skymap : lsst.skymap.SkyMap
#         The LSST SkyMap object.
#     output_path : str or Path
#         Destination path for the output YAML file.
#     inner : bool, optional
#         If True, write inner polygons. If False, write outer polygons. Default is True.
#     """
#     import lsst.geom as geom
#     from lsst.sphgeom import Box
#     from skymap_to_poly_coords.geometry import unit_vector3d_to_radec, box_to_convex_polygon
#     import yaml

#     out = {"poles": [], "rings": []}
#     total_tracts = len(skymap)
#     ring_counts = skymap._ringNums
#     ring_start_index = 0

#     for tract_id in [0, total_tracts - 1]:
#         tract = skymap[tract_id]
#         poly = tract.inner_sky_region if inner else tract.outer_sky_polygon
#         if isinstance(poly, Box):
#             poly = box_to_convex_polygon(poly)

#         radec_poly = [unit_vector3d_to_radec(vec) for vec in poly.getVertices()]
#         ra_list = [round(pt[0] % 360.0, 12) for pt in radec_poly]

#         # Start from bottom-left corner RA
#         bottom_left_ra = round(radec_poly[0][0] % 360.0, 12)
#         tract_entry = {"id": tract_id, "ra": [bottom_left_ra], "type": "pole"}

#         out["poles"].append(tract_entry)

#     for ring_id, n_tracts in enumerate(ring_counts):
#         ring_entry = {"dec_bounds": None, "tracts": []}

#         for i in range(ring_start_index, ring_start_index + n_tracts):
#             tract = skymap[i]
#             tract_id = tract.getId()

#             poly = tract.inner_sky_region if inner else tract.outer_sky_polygon
#             if isinstance(poly, Box):
#                 poly = box_to_convex_polygon(poly)

#             radec_poly = [unit_vector3d_to_radec(vec) for vec in poly.getVertices()]
#             ra_list = [pt[0] for pt in radec_poly]
#             dec_list = [pt[1] for pt in radec_poly]

#             unique_decs = sorted({round(d, 12) for d in dec_list})
#             if len(unique_decs) > 2:
#                 raise ValueError(f"⚠️ Tract {tract_id} has >2 unique Decs: {unique_decs}")

#             if ring_entry["dec_bounds"] is None:
#                 ring_entry["dec_bounds"] = [min(unique_decs), max(unique_decs)]

#             rounded_ras = [round(ra % 360.0, 12) for ra in ra_list]
#             unique_ras = sorted(set(rounded_ras))

#             bottom_left_ra = round(radec_poly[0][0] % 360.0, 12)

#             if len(unique_ras) == 1:
#                 tract_entry = {"id": tract_id, "ra": [bottom_left_ra], "type": "pole"}
#             elif len(unique_ras) == 2:
#                 tract_entry = {"id": tract_id, "ra": unique_ras}
#             elif len(unique_ras) in (3, 4):
#                 # todo : in the future, could be nice to re-verify that these are already in CCW order
#                 start_idx = rounded_ras.index(bottom_left_ra)
#                 ccw_ras = [rounded_ras[(start_idx + j) % 4] for j in range(4)]
#                 tract_entry = {"id": tract_id, "ra": ccw_ras}
#             else:
#                 raise ValueError(f"Tract {tract_id} has unexpected number of unique RAs: {unique_ras}")

#             ring_entry["tracts"].append(tract_entry)

#         out["rings"].append(ring_entry)
#         ring_start_index += n_tracts

#     with open(output_path, "w") as f:
#         yaml.dump(out, f, sort_keys=False)

#     print(f"✅ Ring-optimized polygons written to {output_path}")


In [156]:
def check_dec_bounds(expected_lower, expected_upper, tract_id, inner, skymap):
    tract_info = skymap[tract_id]
    tract_poly = tract_info.inner_sky_region if inner else tract_info.outer_sky_polygon

    # Inner polygon.
    if isinstance(tract_poly, Box): 
        tract_lower = tract_poly.getLat().getA().asDegrees()
        tract_upper = tract_poly.getLat().getB().asDegrees()
        # Flip if needed.
        if tract_lower > tract_upper:
            tract_lower, tract_upper = tract_upper, tract_lower

    # Outer polygon.
    else: 
        print(f"  Tract {tract_id} is a ConvexPolygon - this section is not implemented yet.")

    # Compare with expected bounds.
    if not (math.isclose(tract_lower, expected_lower, abs_tol=1e-6) and
            math.isclose(tract_upper, expected_upper, abs_tol=1e-6)):
        raise ValueError(f"    Tract {tract_id} bounds ({tract_lower:2f}, {tract_upper:2f}); "
                         f"Ring bounds ({expected_lower:2f}, {expected_upper:2f})")

In [157]:
def check_ra_interval(expected_interval, tract_id, inner, skymap):
    # todo rename lower, upper to left, right
    tract_info = skymap[tract_id]
    tract_poly = tract_info.inner_sky_region if inner else tract_info.outer_sky_polygon

    # Inner polygon.
    if isinstance(tract_poly, Box): 
        tract_left = tract_poly.getLon().getA().asDegrees()
        tract_right = tract_poly.getLon().getB().asDegrees()
        # Handle RA wrap-around.
        if abs(tract_left - tract_right) > 180:
            if tract_left > tract_right:
                # Wrap around case, e.g., lower = 350, upper = 10
                tract_right += 360
            else:
                # Normal case, e.g., lower = 10, upper = 350
                tract_left += 360

        # Calculate RA interval.
        tract_interval = abs(tract_right - tract_left)

    # Outer polygon.
    else: 
        print(f"  Tract {tract_id} is a ConvexPolygon - this section is not implemented yet.")

    # Compare with expected interval.
    if not math.isclose(tract_interval, expected_interval, abs_tol=1e-6):
        raise ValueError(f"    Tract {tract_id} interval {tract_interval:2f}; "
                         f"Expected interval {expected_interval:2f}")

In [128]:
def radians_to_degrees(radians):
    """Convert radians to degrees."""
    return radians * (180.0 / math.pi)

In [168]:
def write_ring_optimized_polygons_ra_dec(skymap, output_path, inner=True, skymap_name=None):
    # Psuedo-code for writing ring-optimized polygons in RA/Dec format to a YAML file.
    from skymap_to_poly_coords.geometry import unit_vector3d_to_radec, box_to_convex_polygon

    ring_nums = skymap._ringNums
    ring_size = math.pi / (len(ring_nums) + 1)
    
    # Initialize the output structure.
    out = {"metadata": {}, "poles": [], "rings": []}

    # Add the metadata.
    out["metadata"]["skymap_name"] = skymap_name
    out["metadata"]["inner_polygons"] = inner
    out["metadata"]["ra_start"] = skymap.config.raStart

    # Handle the poles first.


    # Add the rings.
    tract_counter = 1  # Start with 1 since 0 is the south pole.
    for ring, num_tracts in enumerate(ring_nums):
        print(f"Processing ring {ring} with {num_tracts} tracts...")

        # Get the declination bounds for the ring.
        dec = ring_size * (ring + 1) - 0.5 * math.pi
        start_dec = radians_to_degrees(dec - 0.5 * ring_size)
        stop_dec = radians_to_degrees(dec + 0.5 * ring_size)

        # Optional: check the calculated declination with the tracts in the ring.
        for tract_id in range(tract_counter, tract_counter + num_tracts):
            check_dec_bounds(start_dec, stop_dec, tract_id, inner, skymap)
            
        # Get the RA interval for the ring. 
        ra_interval = 360.0 / num_tracts  # degrees

        # Optional: check the RA interval.
        for tract_id in range(tract_counter, tract_counter + num_tracts):
            check_ra_interval(ra_interval, tract_id, inner, skymap)

        # Initialize the ring entry.
        ring_entry = {
            "ring": ring,
            "num_tracts": num_tracts,
            "dec_bounds": [round(start_dec, 10), round(stop_dec, 10)],
            "ra_interval": round(ra_interval, 10),
        }

        # Iterate the tract counter.
        tract_counter += num_tracts

        # Temporary: just break after 10 rings for testing.
        if ring >= 10:
            break

    # Record the output.
    # with open(output_path, "w") as f:
    #     yaml.dump(out, f, sort_keys=False)
    print(out)

In [170]:
write_ring_optimized_polygons_ra_dec(
    lsst_skymap, 
    inner_poly_ring_optimized_path, 
    inner=True, 
    skymap_name="lsst_v1"
)

Processing ring 0 with 10 tracts...
Processing ring 1 with 16 tracts...
Processing ring 2 with 22 tracts...
Processing ring 3 with 29 tracts...
Processing ring 4 with 35 tracts...
Processing ring 5 with 41 tracts...
Processing ring 6 with 47 tracts...
Processing ring 7 with 53 tracts...
Processing ring 8 with 60 tracts...
Processing ring 9 with 66 tracts...
Processing ring 10 with 72 tracts...
{'metadata': {'skymap_name': 'lsst_v1', 'inner_polygons': True, 'ra_start': 0.0}, 'poles': [], 'rings': []}


### Read back in

In [ ]:
import yaml
from pathlib import Path

class RingOptimizedPolygonReader:
    """Reader for ring-optimized RA/Dec polygon files."""

    def __init__(self, yaml_path):
        """
        Initialize the reader and build ring structure metadata.

        Parameters
        ----------
        yaml_path : str or Path
            Path to the YAML file storing the ring-optimized polygons.
        """
        self.yaml_path = Path(yaml_path)
        self._ring_sizes = []  # Number of tracts in each ring
        self._pole_tracts = []  # Typically [0, N]
        self._load_metadata()

    def _load_metadata(self):
        """Load basic metadata like ring sizes and pole IDs."""
        with open(self.yaml_path, "r") as f:
            self._data = yaml.safe_load(f)

        # Store ring sizes
        self._ring_sizes = [len(ring["tracts"]) for ring in self._data.get("rings", [])]

        # Store pole tracts
        self._pole_tracts = [t["id"] for t in self._data.get("poles", [])]

    def _get_ring_and_local_idx(self, tract_id):
        """Given a tract ID, return (ring_idx, local_idx) or None if it's a pole."""
        if tract_id in self._pole_tracts:
            return None, None

        offset = 0
        for ring_idx, size in enumerate(self._ring_sizes):
            if offset <= tract_id < offset + size:
                local_idx = tract_id - offset
                return ring_idx, local_idx
            offset += size

        raise ValueError(f"Tract ID {tract_id} not found in ring structure.")

    def get_polygon_vertices(self, tract_id):
        """
        Lazily load the polygon vertices for a given tract.

        Parameters
        ----------
        tract_id : int
            The tract ID to load.

        Returns
        -------
        list of [float, float]
            List of (RA, Dec) vertices in degrees.
        """
        with open(self.yaml_path, "r") as f:
            data = yaml.safe_load(f)

        if tract_id in self._pole_tracts:
            for pole in data["poles"]:
                if pole["id"] == tract_id:
                    dec = pole["dec"]
                    ra_list = pole["ra"]
                    return [[ra, dec] for ra in ra_list]
            raise ValueError(f"Pole tract {tract_id} not found.")

        ring_idx, local_idx = self._get_ring_and_local_idx(tract_id)
        ring = data["rings"][ring_idx]
        tract = ring["tracts"][local_idx]

        dec_min, dec_max = ring["dec_bounds"]
        ra_list = tract["ra"]

        # Infer vertex layout — always CCW, starting from bottom left
        if len(ra_list) == 1:
            return [[ra_list[0], dec_min],
                    [ra_list[0], dec_max]]
        elif len(ra_list) == 2:
            return [[ra_list[0], dec_min],
                    [ra_list[1], dec_min],
                    [ra_list[1], dec_max],
                    [ra_list[0], dec_max]]
        elif len(ra_list) == 4:
            return [[ra_list[0], dec_min],
                    [ra_list[1], dec_min],
                    [ra_list[2], dec_max],
                    [ra_list[3], dec_max]]
        else:
            raise ValueError(f"Unexpected number of RAs for tract {tract_id}: {ra_list}")



In [ ]:
poly_reader = RingOptimizedPolygonReader(inner_poly_ring_optimized_path)

In [ ]:
poly_reader.get_polygon_vertices(42)

In [ ]:
# todo : handle poly_reader.get_polygon_vertices(0)